## Genes

In [1]:
import pandas as pd 
data  = pd.read_csv('gene_high_throughput_sequencing.csv')
data.head()
#len(data)

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


### Часть 1. Применение критерия Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках.

In [45]:
def write_answer(value, name):
    with open(name, 'w+') as file:
        file.write(str(value))
write_answer(10343483, 'proba_pera.txt')        

In [55]:
import numpy as np
import pandas as pd
import matplotlib as mp
%matplotlib inline
import scipy.stats as stats
pvalues = {}
for i in data.columns[2:]:
    pvalues[i] = stats.ttest_ind( data.ix[data['Diagnosis']=='early neoplasia', i], data.ix[data['Diagnosis']=='normal', i])[1]


In [46]:
count = 0
for i in pvalues.keys():
    if pvalues[i]<0.05:
        count+=1        
print 'Count of different values for genes:', count  
print len(pvalues)
write_answer(count,'genes_1_1.txt')

Count of different values for genes: 1583
15748


In [47]:
pvalues2 = {}
for i in data.columns[2:]:
    pvalues2[i] = stats.ttest_rel(data.ix[data['Diagnosis']=='early neoplasia', i], data.ix[data['Diagnosis']=='cancer', i])[1]
count = 0
for i in pvalues2.keys():
    if pvalues2[i]<0.05:
        count+=1        
print 'Count of different values for genes:', count  
print len(pvalues2)
write_answer(count,'genes_1_2.txt')

Count of different values for genes: 3529
15748


### Часть 2: поправка методом Холма

alpha=0.25 из-за поправки Бонферони, потому что мы имеем 2 набора уровней значимости, и мы их сравниваем по отдельности

In [49]:
from statsmodels.sandbox.stats.multicomp import multipletests 
reject, p_corrected, a1, a2 = multipletests(map(lambda keys:pvalues[keys], data.columns[2:]), 
                                            alpha = 0.025, 
                                            method = 'holm') 
reject2, p_corrected2, a1_2, a2_2 = multipletests(map(lambda keys:pvalues2[keys], data.columns[2:]), 
                                            alpha = 0.025, 
                                            method = 'holm') 
print sum(reject)
print sum(reject2)



2
85


In [50]:
def fold_change(t, c):
    if t>=c:
        return t/c
    else:
        return -c/t
    
print '1st experiment'
practical_val_arr = []
for i,j in zip(reject, data.columns):
    if i:
        practical_val_arr.append( fold_change(data.ix[data['Diagnosis']=='normal',j].mean(),\
                              data.ix[data['Diagnosis']=='early neoplasia',j].mean()))
for i in practical_val_arr:
    if np.abs(i)>1.5:
        print i
practical_val_arr = []        
print '2nd experiment'

for i,j in zip(reject2, data.columns):
    if i:
        practical_val_arr.append(fold_change(data.ix[data['Diagnosis']=='early neoplasia',j].mean(),\
                              data.ix[data['Diagnosis']=='cancer',j].mean()))
for i in practical_val_arr:
    if np.abs(i)>1.5:
        print i
        
write_answer(0,'genes_2_1.txt')
write_answer(4,'genes_2_2.txt')

1st experiment
2nd experiment
-2.29203566805
1.53339432664
1.6628886446
2.25183903815


### Часть 3: поправка методом Бенджамини-Хохберга

In [51]:
from statsmodels.sandbox.stats.multicomp import multipletests 
reject3, p_corrected3, a1_3, a2_3 = multipletests(map(lambda keys:pvalues[keys], data.columns[2:]), 
                                            alpha = 0.025, 
                                            method = 'fdr_bh') 
reject4, p_corrected4, a1_3, a2_4 = multipletests(map(lambda keys:pvalues2[keys], data.columns[2:]), 
                                            alpha = 0.025, 
                                            method = 'fdr_bh') 
print sum(reject3)
print sum(reject4)



4
908


In [52]:
print '1st experiment'
practical_val_arr = []
for i,j in zip(reject3, data.columns):
    if i:
        practical_val_arr.append( fold_change(data.ix[data['Diagnosis']=='normal',j].mean(),\
                              data.ix[data['Diagnosis']=='early neoplasia',j].mean()))
for i in practical_val_arr:
    if np.abs(i)>1.5:
        print i
practical_val_arr = []        
print '2nd experiment'

for i,j in zip(reject4, data.columns):
    if i:
        practical_val_arr.append(fold_change(data.ix[data['Diagnosis']=='early neoplasia',j].mean(),\
                              data.ix[data['Diagnosis']=='cancer',j].mean()))
count=0        
for i in practical_val_arr:
    if np.abs(i)>1.5:
        count+=1
print count  
write_answer(0,'genes_3_1.txt')
write_answer(count,'genes_3_2.txt')

1st experiment
2nd experiment
84
